In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import heapq

%matplotlib inline 

In [53]:
uniq_id_cust= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_customers_dataset.csv')
orders= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_orders_dataset.csv')
items= pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_order_items_dataset.csv')

In [54]:
#Задание 3. По каждому товару определить, в какой день недели товар чаще всего покупается. 

In [55]:
#объединим заказы и продукты для определение времени
join_table2 = items.merge(orders, how = 'left', on ='order_id')

In [56]:
join_table2.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [57]:
# привели к формату даты время подтверждения покупки
join_table2 ['buying_day'] = pd.to_datetime(join_table2 ['order_approved_at'])

In [58]:
# привели к дню недели 
join_table2 ['buying_day'] = join_table2.buying_day.dt.strftime('%A')

In [63]:
# отберем все продажи по всем дням недели,кроме недоступных товаров и неоплаченных
all_sales_items = join_table2\
    .query('order_status == "processing" or order_status == "shipped" or order_status == "delivered"')\
    .groupby (['product_id','buying_day'])\
    .agg ({'order_id': 'count'})\
    .rename(columns={'order_id': 'count_orders'})\
    .reset_index()\
    .sort_values(['product_id', 'count_orders'], ascending = False)\
    .groupby (['product_id','buying_day'])\
    .agg ({'count_orders': 'max'})\
    .sort_values('count_orders', ascending = False)

In [64]:
# В этой таблице указаны покупки всех товаров по всем дням недели
all_sales_items

count_orders
product_id                       buying_day              
aca2eb7d00ea1a7b8ebd4e68314663af Tuesday              119
53b36df67ebb7c41585e8d54d6772e08 Tuesday              105
aca2eb7d00ea1a7b8ebd4e68314663af Friday                96
                                 Wednesday             89
422879e10f46682990de24d770e7f83d Tuesday               89
...                                                   ...
64051126507bdb875c1baf5b3b5a9a98 Tuesday                1
                                 Wednesday              1
6405e17a3e8718d5316c5d644320f253 Saturday               1
                                 Sunday                 1
fffe9eeff12fcbd74a2f2b007dde0c58 Wednesday              1

[60574 rows x 1 columns]

In [12]:
# Ответ: Выше приведена таблица с данными по частоте покупок для каждго товара (с указанием дня недели)